In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


/data/hieunm/anaconda3/envs/wids-env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_df =  pd.read_csv('sub_train.csv')
test_df  =  pd.read_csv('sub_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

startdate
climateregions__climateregion


In [3]:
test_df[test_df['climateregions__climateregion'] == 0]

,startdate,contest-wind-h500-14d__wind-hgt-500,contest-slp-14d__slp,nmme-tmp2m-34w__ccsm3,elevation__elevation,lon,contest-prwtr-eatm-14d__prwtr,lat,climateregions__climateregion,contest-pres-sfc-gauss-14d__pres,season_sin,day_of_year_sin,contest-precip-14d__precip,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-prate-34w__cfsv2,nmme-prate-34w__nasa,nmme-prate-56w__gfdlflora,wind-uwnd-250-2010-7,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-34w__nmmemean,nmme0-prate-34w__ccsm30,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,nmme0-prate-56w__cfsv20,nmme-prate-34w__cancm3,contest-rhum-sig995-14d__rhum,nmme-prate-34w__gfdlflorb,wind-hgt-850-2010-4,contest-wind-vwnd-250-14d__wind-vwnd-250,wind-hgt-100-2010-2,wind-uwnd-250-2010-18,wind-hgt-10-2010-5,wind-uwnd-250-2010-15,wind-uwnd-250-2010-4,nmme0-prate-56w__nasa0,nmme0-prate-34w__cfsv20,wind-vwnd-250-2010-10,contest-wind-h10-14d__wind-hgt-10,wind-uwnd-925-2010-15,wind-vwnd-250-2010-3,nmme-prate-34w__cancm4,sst-2010-4,nmme0-prate-56w__ccsm30,wind-uwnd-250-2010-16,nmme0-prate-34w__gfdl0,nmme0-prate-56w__cancm40,sst-2010-1,sst-2010-3,wind-uwnd-250-2010-14,nmme0-prate-34w__gfdlflora0,nmme-prate-34w__gfdl,wind-hgt-850-2010-9,wind-vwnd-250-2010-1,sst-2010-5,cancm30,nmme-prate-34w__ccsm4,nmme0-prate-34w__nasa0,wind-hgt-500-2010-9,nmme0-prate-34w__cancm30,wind-vwnd-250-2010-13,index
0,2022-11-01,5916.86,101441.09,27.37,200.0,0.833333,40.29,0.000000,0,98734.98,-1.0,-8.587640e-01,34.85,1.14,32.67,28.50,30.67,-40.56,3.12,25.09,12.24,1549.82,-4.38,29.50,20.01,72.04,33.57,1023.51,-4.82,-6044.99,142.32,7408.07,-75.49,221.61,19.67,40.72,72.60,31233.02,57.53,-210.45,21.62,41.38,16.12,-13.39,8.88,11.48,345.33,-19.05,2.89,19.03,15.33,-337.74,-176.44,12.53,29.44,29.60,24.46,-529.25,15.81,57.01,375734
1,2022-11-02,5917.85,101449.71,27.37,200.0,0.833333,40.15,0.000000,0,98745.78,-1.0,-8.498171e-01,27.46,0.55,32.67,28.50,30.67,-41.01,3.22,25.09,12.24,1550.55,-4.50,29.50,20.01,71.93,33.57,815.95,-4.34,-6152.43,140.27,6913.60,-70.65,221.58,19.67,40.72,67.94,31230.63,45.23,-211.43,21.62,39.57,16.12,-8.10,8.88,11.48,344.50,-19.20,-7.43,19.03,15.33,-243.39,-158.62,12.27,29.44,29.60,24.46,-438.92,15.81,56.24,375735
2,2022-11-03,5918.56,101446.14,27.37,200.0,0.833333,40.07,0.000000,0,98743.07,-1.0,-8.406184e-01,24.65,0.12,32.67,28.50,30.67,-51.24,3.42,25.09,12.24,1550.16,-4.61,29.50,20.01,72.43,33.57,619.22,-3.62,-6296.21,137.79,6445.81,-66.89,221.06,19.67,40.72,66.51,31225.26,33.94,-209.54,21.62,37.79,16.12,-11.03,8.88,11.48,343.40,-19.35,-12.71,19.03,15.33,-51.71,-138.70,11.94,29.44,29.60,24.46,-325.51,15.81,45.71,375736
3,2022-11-04,5920.30,101443.27,27.37,200.0,0.833333,40.00,0.000000,0,98748.59,-1.0,-8.311706e-01,22.43,0.26,32.67,28.50,30.67,-61.72,3.82,25.09,12.24,1550.15,-4.49,29.50,20.01,71.88,33.57,459.28,-2.80,-6417.62,134.52,5895.94,-73.97,218.18,19.67,40.72,75.15,31220.87,29.31,-202.11,21.62,35.93,16.12,-23.55,8.88,11.48,341.91,-19.38,-17.76,19.03,15.33,191.32,-122.68,11.34,29.44,29.60,24.46,-274.45,15.81,41.89,375737
4,2022-11-05,5923.03,101448.09,27.37,200.0,0.833333,39.80,0.000000,0,98761.84,-1.0,-8.214766e-01,16.31,0.65,32.67,28.50,30.67,-66.26,3.91,25.09,12.24,1551.19,-4.34,29.50,20.01,71.18,33.57,326.11,-2.59,-6479.77,126.65,5383.52,-81.71,214.43,19.67,40.72,91.43,31214.54,30.52,-187.25,21.62,33.97,16.12,-26.73,8.88,11.48,340.22,-19.27,-18.64,19.03,15.33,290.21,-113.51,10.56,29.44,29.60,24.46,-279.28,15.81,68.04,375738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2191,2022-12-27,5835.23,101487.31,14.30,1000.0,0.433333,14.83,0.227273,0,90934.84,0.0,-6.880243e-02,1.94,12.61,17.28,6.76,16.19,11.55,-0.26,8.18,4.97,1530.34,-0.82,13.37,4.20,38.29,4.86,83.41,-1.33,-2239.27,-103.19,8744.09,55.53,-289.95,9.17,11.79,-1.09,30864.27,4.48,149.53,3.69,-39.74,1.81,1.38,6.22

In [4]:
test_df['index']

0        375734
1        375735
2        375736
3        375737
4        375738
          ...  
31349    407083
31350    407084
31351    407085
31352    407086
31353    407087
Name: index, Length: 31354, dtype: int64

In [5]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
region = ["climateregions__climateregion"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target and c not in region]
print("Main features:", len(main_attrs))

Main features: 58


In [6]:
train_df_list = []
test_df_list = []

for i in range(15):
    train_df_list.append(train_df[train_df[region[0]] == i])
    test_df_list.append(test_df[test_df[region[0]] == i])    

In [7]:
for i in range(15):
    output_df = pd.read_csv(f'./per_sub_{i}.csv')

In [9]:
index = 1
output_df = pd.read_csv(f'./per_sub_{index}.csv')
# output_df['index'] = test_df_list[index]['index'].tolist()

In [10]:
best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values
merged_df = best_df.merge(output_df, how='left', on='index')

In [11]:
merged_df['contest-tmp2m-14d__tmp2m_y'].isnull().sum()

19703

In [12]:
import math
t = merged_df.values
for i in range(t.shape[0]):
    if not math.isnan(t[i][2]):
        print(t[i][0] - t[i][2])

-0.06029626259683596
0.01762141594433686
0.1321856345296588
0.15461216386842125
0.1731666435935466
0.08653522427490756
0.285441504614262
0.377029854327283
0.28293444305426974
0.4390294685310394
0.4711927596650938
0.2532665071403706
0.3488652831124597
0.38404882275214547
0.6597900342416558
0.6587369775121488
0.696903696777678
0.8171455609445957
0.804890166567656
0.4878418791642858
0.3544215951658316
0.47062533930723305
0.5734101341087907
0.49998458990068073
0.5803541265507768
0.6039963903551246
0.4286754608469039
0.029875808722241715
0.06843197329537887
0.01596248784049692
-0.23516395981821603
-0.24360274979072827
-0.39047438245324173
-0.5030279191449196
-0.45254482685740527
-0.17861803815967647
-0.2533263553545595
-0.24414414272842677
-0.1975066696665948
-0.32226946902614984
-0.48917201840074753
-0.5441762691146792
-0.7361286923241153
-0.6202236425102292
-0.2196651209099123
-0.11643012930606744
-0.015090612785986224
-0.17309421720031182
-0.2745249611962848
0.041458143382936186
0.174530

In [13]:
import math
def preprocess_result(x):
    r1, r2 = x
    if math.isnan(r2):
        return r1
    return r2 * 0.1 + r1 * 0.9

In [14]:
result = merged_df[['contest-tmp2m-14d__tmp2m_x', 'contest-tmp2m-14d__tmp2m_y']].apply(preprocess_result, axis=1)
df = pd.read_csv('./sample_solution.csv')
df['contest-tmp2m-14d__tmp2m'] = result
df.to_csv('submission_per_sub_o.csv', index=False)

from numpy import dot
from numpy.linalg import norm
y_best = best_df[target[0]].values
cos_sim = dot(y_best, result)/(norm(y_best)*norm(result))
print("cos_sim with best submission:", cos_sim)

cos_sim with best submission: 0.9999989577821907
